<a href="https://colab.research.google.com/github/ravindrakush11/AI-Agents/blob/main/hf_agent_orchestrate_a_multi_agent_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Orchestrate a multi-agent system:--** a multi-agent web browser: an agentic system with several agents collaborating to solve problems using the web!

In [ ]:
!pip install smolagents[toolkit] --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.2/484.2 kB 7.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.4 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [ ]:
from huggingface_hub import login
login()

In [ ]:
model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"

## Create a Web search tool

In [ ]:
import re
import requests
from markdownify import markdownify
from requests.exceptions import RequestException
from smolagents import tool

@tool
def visit_webpage(url: str) -> str:
  """Visits a webpage at the given URL and returns its content as markdown string.

  Args:
      url: The URL of the webpage to visit.

  Returns:
      The content of the webpage converted to Markdown, or an error message if the request fails.
  """

  try:
    response = requests.get(url)
    response.raise_for_status()

    markdown_content = markdownify(response.text).strip()

    markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

    return markdown_content

  except RequestException as e:
    return f"Error fetching the webpage: {str(e)}"
  except Exception as e:
    return f"An unexpected error occurred: {str(e)}"

In [ ]:
print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[500:])

ct us](//en.wikipedia.org/wiki/Wikipedia:Contact_us "How to contact Wikipedia")

Contribute

* [Help](/wiki/Help:Contents "Guidance on how to use and edit Wikipedia")
* [Learn to edit](/wiki/Help:Introduction "Learn how to edit Wikipedia")
* [Community portal](/wiki/Wikipedia:Community_portal "The hub for editors")
* [Recent changes](/wiki/Special:RecentChanges "A list of recent changes to Wikipedia [r]")
* [Upload file](/wiki/Wikipedia:File_upload_wizard "Add images or other media for use on Wikipedia")
* [Special pages](/wiki/Special:SpecialPages)

[![](/static/images/icons/wikipedia.png)

![Wikipedia](/static/images/mobile/copyright/wikipedia-wordmark-en.svg)
![The Free Encyclopedia](/static/images/mobile/copyright/wikipedia-tagline-en.svg)](/wiki/Main_Page)

[Search](/wiki/Special:Search "Search Wikipedia [f]")

Search

Appearance

* [Donate](https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_campaign=en.wikipedia.org&uselang=en)
* [Create account](/w/index.php?

## Build our multi-agent system

In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    InferenceClientModel,
    WebSearchTool,
    LiteLLMModel,
)

model = InferenceClientModel(model_id=model_id)

web_agent = ToolCallingAgent(
    tools = [WebSearchTool(), visit_webpage],
    model = model,
    max_steps = 10,
    name = "web_search_agent",
    description= "Runs web searches for you.",
)

In [ ]:
manager_agent = CodeAgent(
    tools = [],
    model = model,
    managed_agents = [web_agent],
    additional_authorized_imports=['time', 'numpy', 'pandas'],   # just in case the agent needs these packages.
)

In [ ]:
# Run the system
answer = manager_agent.run("If LLM training continues to scale up at the current rhythm until 2030, what would be the electric power in GW required to power the biggest training runs by 2030? What would that correspond to, compared to some countries> Please provide a source for any numbers used.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ If LLM training continues to scale up at the current rhythm until 2030, what would be the electric power in GW  │
│ required to power the biggest training runs by 2030? What would that correspond to, compared to some countries> │
│ Please provide a source for any numbers used.                                                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
401 Client Error: Unauthorized for url: 
https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: 
Root=1-682813ab-474d689b7d3dc8737d5ef5f7;f63e0eaa-0174-4ea0-b894-b4cc70c3012a)

Invalid username or password.

[Step 1: Duration 0.34 seconds]

AgentGenerationError: Error in generating model output:
401 Client Error: Unauthorized for url: https://router.huggingface.co/hf-inference/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-682813ab-474d689b7d3dc8737d5ef5f7;f63e0eaa-0174-4ea0-b894-b4cc70c3012a)

Invalid username or password.